In [1]:
pip install python-gedcom

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tabulate


Note: you may need to restart the kernel to use updated packages.


In [3]:
### gedcom library

from gedcom.parser import Parser
from gedcom.element.individual import IndividualElement
from gedcom.element.family import FamilyElement
import pandas as pd
import numpy as np
from tabulate import tabulate
import datetime
from datetime import date

import warnings
warnings.filterwarnings("ignore")

In [4]:
def calculateAge(birthDate, deathDate):
    
      age = deathDate.year - birthDate.year - ((deathDate.month, deathDate.day) <
            (birthDate.month, birthDate.day))
         
 
      return age

In [5]:
def getIndividuals(file):
    
    ### dataframe to store individuals details
    individuals = pd.DataFrame(columns=['ID','Name','Gender','Birthday','Age','Alive','Death','Child','Spouse'])
    
    ### Parse the gedcom file
    parser = Parser()
    parser.parse_file(file)
    
    ### Iterate through each element and pull an individual data based on level and tag conditions
    for ele in parser.get_element_list():
        
        if(IndividualElement.is_individual(ele) == True):
            
            famSpouses = set()
            famChilds = set()            
            id = ele.get_pointer().replace('@', '')
            deathDate = 'NA'
            alive = 'True'
            
            ### For each individual element, below extracting the personal data
            child_elements = IndividualElement.get_child_elements(ele)
            for child in child_elements:
                if(child.get_tag() == 'NAME'):
                    name = child.get_value()
                    # print(name)
                elif(child.get_tag() == 'SEX'):
                    gender = child.get_value()
                    # print(gender)
                elif(child.get_tag() == 'BIRT'):
                    birthDate = datetime.datetime.strptime(child.get_child_elements()[0].get_value(),'%d %b %Y').strftime('%Y-%m-%d')
                    # print(birthDate)
                    age = calculateAge(date(int(birthDate.split('-')[0]),int(birthDate.split('-')[1]),int(birthDate.split('-')[2])), 
                            date.today())
                elif(child.get_tag() == 'FAMS'):
                    famSpouses.add(child.get_value().replace('@', ''))
                    # print('Spouse ', famSpouses)
                elif(child.get_tag() == 'FAMC'):
                    famChilds.add(child.get_value().replace('@', ''))
                    # print('Childs ', famChilds)
                elif(child.get_tag() == 'DEAT'):
                    deathDate = datetime.datetime.strptime(child.get_child_elements()[0].get_value(),'%d %b %Y').strftime('%Y-%m-%d')
                    # print(deathDate)
                    alive = 'False'
                    age = calculateAge(date(int(birthDate.split('-')[0]),int(birthDate.split('-')[1]),int(birthDate.split('-')[2])), 
                           date(int(deathDate.split('-')[0]),int(deathDate.split('-')[1]),int(deathDate.split('-')[2])))
                    
        
            # print('*********')  
        ### Appending each individual to dataframe
            row = {'ID': id, 'Name': name, 'Gender': gender, 
                'Birthday': birthDate, 'Age': age, 'Alive': alive,
                'Death': deathDate, 'Child': famChilds, 'Spouse': famSpouses}
            
            individuals = individuals.append(row, ignore_index = True)
            
    return individuals